# 2021-02-09 Dataset Maker

Hi! I am  [Jean-Nicolas Jérémie](https://github.com/JNJER) and the goal is to create a dataset of images containening a folder with animated and another with non animated. Based on the prediction of deep convolutional neuronal networks like `VGG16`, `ResNet101`, `AlexNet`, `MobileNet` on the [Imagenet](http://image-net.org/) dataset wich allows to work on naturals images for $K = 1000$ labels or even a Single Shot Detection network (`SSD` based on a VGG architecture implemented from a very neat [tutorial](https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Object-Detection.git)) working on the [The PASCAL VOC project](http://host.robots.ox.ac.uk/pascal/VOC/) dataset wich allows to work on naturals images for $K = 20$ labels. Here we decide to try the performances on dataset like [PerrinetBednar15](https://github.com/laurentperrinet/PerrinetBednar15) or [Object and Semantic Images and Eye-tracking (OSIE) data set](https://github.com/NUS-VIP/predicting-human-gaze-beyond-pixels.git). 

In this notebook, I will use the [Pytorch](https://pytorch.org/) library for running the networks and the [pandas](https://pandas.pydata.org/docs/getting_started/index.html) library to collect and display the results. This notebook was done during a master 2 internship at the Neurosciences Institute of Timone (INT) under the supervision of [Laurent PERRINET](https://laurentperrinet.github.io/). It is curated in the following [github repo](https://github.com/JNJER/2021-01-11_fast_and_curious.git).

In [1]:
%matplotlib inline
%mkdir -p DCNN_dataset_maker

## Importing libraries; definition of the dataset
Our coding strategy is to build up a small libray as a package of scripts in the `DCNN_benchmark` folder and to run all calls to that library from this notebook.


In [2]:
%%writefile DCNN_dataset_maker/init.py

# Importing libraries
import os
from PIL import Image, ImageDraw, ImageFont
from time import strftime, gmtime
import time 
import json
import time 
import numpy as np
import imageio
from utils import *
from numpy import random
import argparse
import cv2 
import os.path as osp
import shutil
from torchvision.datasets import ImageFolder

# to plot &  display
import matplotlib.pyplot as plt
def pprint(message):
    print('-'*len(message))
    print(message)
    print('-'*len(message))

# parse the root to the init module
def arg_parse(): 
    parser = argparse.ArgumentParser(description='DCNN_training_benchmark/init.py set root')
    parser.add_argument("--path_in", dest = 'path_in', help = 
                        "Set the Directory containing images to perform detection upon",
                        default = '/Users/jjn/Desktop/test', type = str)
    parser.add_argument("--path_out", dest = 'path_out', help = 
                        "Set the Directory containing images to perform detection upon",
                        default = '/Users/jjn/Desktop/test_1', type = str)
    parser.add_argument("--HOST", dest = 'HOST', help = 
                    "Set the name of your machine",
                    default = os.uname()[1], type = str)
    parser.add_argument("--datetag", dest = 'datetag', help = 
                    "Set the datetag of the result's file",
                    default = strftime("%Y-%m-%d", gmtime()), type = str)
    parser.add_argument("--image_size", dest = 'image_size', help = 
                    "Set the image_size of the input",
                    default = 256)
    parser.add_argument("--image_size_SSD", dest = 'image_size_SSD', help = 
                    "Set the image_size of the input",
                    default = 300)
    parser.add_argument("--imagenet_label_root", dest = 'imagenet_label_root', help = 
                        "Set the Directory containing imagenet labels",
                        default = 'imagenet_classes.txt', type = str)
    parser.add_argument("--min_score", dest = "min_score", help = 
                        "minimum threshold for a detected box to be considered a match for a certain class",
                        default = 0.6)
    parser.add_argument("--max_overlap", dest = "max_overlap", help = 
                        "maximum overlap two boxes can have so that the one with the lower score is not suppressed via Non-Maximum Suppression (NMS)",
                        default = 0.3)
    parser.add_argument("--checkpoint", dest = 'checkpoint', help = 
                        "weightsfile",
                        default = "checkpoint_ssd300.pth.tar", type = str)
    parser.add_argument("--top_k", dest = 'top_k', help = 
                        "if there are a lot of resulting detection across all classes, keep only the top 'k'",
                        default = 200 , type = str)
    parser.add_argument("--class_crop", dest = 'class_crop', help = 
                        "Select a class to trigger the crop",
                        default = "person", type = str)
    parser.add_argument("--class_roll", dest = 'class_roll', help = 
                        "Select a class to trigger the roll",
                        default = "person", type = str)
    return parser.parse_args()

args = arg_parse()

# SSD variables 
min_score = float(args.min_score)
max_overlap = float(args.max_overlap)
checkpoint = args.checkpoint
top_k = float(args.top_k)
animated= ( 'bird', 'cat', 'cow', 'dog', 'horse', 'person', 'sheep')
class_crop = args.class_crop
class_roll = args.class_roll

# figure's variables
fig_width = 20
phi = (np.sqrt(5)+1)/2 # golden ratio
phi = phi**2
colors = ['b', 'r', 'k','g']

# host & date's variables 
HOST = args.HOST
datetag = args.datetag
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#dataset configuration
image_size = args.image_size # default image resolution
image_size_SSD = args.image_size_SSD # default image resolution
id_dl = ''

path_in = args.path_in 
path_out = args.path_out 
path_out_a = os.path.join(path_out+'/animated')
path_out_b = os.path.join(path_out+'/non_animated')
path_out_crop = os.path.join(path_out+'/crop')
path_out_roll = os.path.join(path_out+'/roll')

imagenet_label_root = args.imagenet_label_root
with open(imagenet_label_root) as f:
    labels = [line.strip() for line in f.readlines()]
labels[0].split(', ')
labels = [label.split(', ')[1].lower().replace('_', ' ') for label in labels]

def is_animated(x):
    if x <=397:
        return 1
    else:
        return
pprint('init.py : Done !')

Overwriting DCNN_dataset_maker/init.py


In [3]:
%run DCNN_dataset_maker/init.py

----------------
init.py : Done !
----------------


In [4]:
if HOST == 'fortytwo':
    do_local = False 
    python_exec = "KMP_DUPLICATE_LIB_OK=TRUE python3"
else :
    do_local =True

## Creating/checking folders to data

In [5]:
scriptname = 'DCNN_dataset_maker/dataset.py'

In [6]:
%%writefile {scriptname}

from init import *

# check if the folder exist
if os.path.isdir(path_out):
    list_dir = os.listdir(path_out)
    if not "animated" in list_dir:
        pprint(f"No existing path match for this folder, creating a folder at {path_out_a}")
        os.makedirs(path_out_a)
    if not "non_animated" in list_dir:
        pprint(f"No existing path match for this folder, creating a folder at {path_out_b}")
        os.makedirs(path_out_b)
    if not "crop" in list_dir:
        pprint(f"No existing path match for this folder, creating a folder at {path_out_crop}")
        os.makedirs(path_out_crop)
    if not "roll" in list_dir:
        pprint(f"No existing path match for this folder, creating a folder at {path_out_roll}")
        os.makedirs(path_out_roll)
    else:
        pprint(f"The folder already exists, it includes: {list_dir}")

# no folder, creating one 
else :
    print(f"No existing path match for this folder, creating a folder at {path_out}")
    os.makedirs(path_out)
    os.makedirs(path_out_a)
    os.makedirs(path_out_b)
    list_dir = os.listdir(path_out)
    pprint(f"Now the folder contains : {os.listdir(path_out)}")

if os.path.isdir(path_in):
    list_dir = os.listdir(path_in)
    pprint(f"The folder already exists, it includes: {list_dir}")
else :
    pprint(f"No existing path match for this folder at {path_in} check your data !")


Overwriting DCNN_dataset_maker/dataset.py


In [7]:
if do_local:
    %run {scriptname}
else:
    !python3 {scriptname}

----------------
init.py : Done !
----------------
---------------------------------------------------------------------------------------------------
No existing path match for this folder, creating a folder at /Users/jjn/Desktop/test_1/non_animated
---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------
No existing path match for this folder, creating a folder at /Users/jjn/Desktop/test_1/non_animated
---------------------------------------------------------------------------------------------------
-----------------------------------------------
The folder already exists, it includes: ['det']
-----------------------------------------------


## Pre-trained network's import

Here we worked on five differents pre-trained networks `Alexnet`, `Mobilenet`, `Resnet101`, `VGG16` & `SSD` :

In [8]:
scriptname = 'DCNN_dataset_maker/models.py'

In [9]:
%%writefile {scriptname}

from DCNN_dataset_maker.init import *

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

# transform function for input's image processing
transform_SSD = transforms.Compose([
    transforms.Resize((int(image_size_SSD),int(image_size_SSD))),      # Resize the image to image_size x image_size pixels size.
    transforms.ToTensor(),       # Convert the image to PyTorch Tensor data type.
    transforms.Normalize(        # Normalize the image by adjusting
    mean=[0.485, 0.456, 0.406],  #  its average and
    std=[0.229, 0.224, 0.225]    #  its standard deviation at the specified values.              
    )])


# transform function for input's image processing
transform = transforms.Compose([
    transforms.Resize((int(image_size))),      # Resize the image to image_size x image_size pixels size.
    transforms.ToTensor(),       # Convert the image to PyTorch Tensor data type.
    transforms.Normalize(        # Normalize the image by adjusting
    mean=[0.485, 0.456, 0.406],  #  its average and
    std=[0.229, 0.224, 0.225]    #  its standard deviation at the specified values.              
    )])

image_dataset = ImageFolder(path_in, transform=transform) # save the dataset
image_dataset_SSD = ImageFolder(path_in, transform=transform_SSD) # save the dataset

# imports networks with weights
models = {} # get model's names

# Load SSD model from the checkpoint
checkpoint = 'checkpoint_ssd300.pth.tar'
try:
    checkpoint = torch.load(checkpoint)
except:
    checkpoint = torch.load(checkpoint, map_location=torch.device('cpu'))
pprint('Loaded SSD model')
model = checkpoint['model'].to(device).eval()


#models['vgg16'] = torchvision.models.vgg16(pretrained=True)
print("Loading pretrained torchvision's model..")
models['alex'] = torchvision.models.alexnet(pretrained=True)
models['vgg'] = torchvision.models.vgg16(pretrained=True)
models['mob'] = torchvision.models.mobilenet_v2(pretrained=True)
models['res'] = torchvision.models.resnext101_32x8d(pretrained=True)
pprint("Loaded!")

for name in models.keys():
    models[name].to(device).eval()

print(datetag, 'Running benchmark on host', HOST, 'with',device)

Overwriting DCNN_dataset_maker/models.py


In [10]:
%run {scriptname}

----------------
init.py : Done !
----------------


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'model.SSD300' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.pooling.MaxPool2d' has changed

----------------
Loaded SSD model
----------------
Loading pretrained torchvision's model..
-------
Loaded!
-------
2021-02-11 Running benchmark on host MacBook-Air-de-jeremie.local with cpu


## Create a Dataset based on a criterion on the Imagenet Dataset (here animated)

In [11]:
scriptname = 'DCNN_classif.py'

In [12]:
%%writefile {scriptname}


#import model's script and set the output file
from DCNN_dataset_maker.models import * 
i = 0 
alive = []
for i_image, (data, label) in enumerate(image_dataset):
    for name in models.keys():
        model = models[name]
        tic = time.time()
        out = model(data.unsqueeze(0).to(device)).squeeze(0)
        percentage = torch.nn.functional.softmax(out, dim=0) * 100
        _, indices = torch.sort(out, descending=True)
        dt = time.time() - tic  
        for idx in indices[:1]:
            if percentage[idx].item() > 25 and  idx <=397:
                alive.append(1)
                #print(image_dataset.imgs[i_image][0], alive)
    if len(alive)>=2 : 
        i+=1
        print(f'The {name} model get {labels[idx]} at {percentage[idx].item():.2f} % confidence in {dt:.3f} seconds') # Affichage du meilleur pourcentage
        shutil.copy(image_dataset.imgs[i_image][0], path_out_a)
    else:
        shutil.copy(image_dataset.imgs[i_image][0], path_out_b)   
    alive = []
print(i)

Overwriting DCNN_classif.py


In [13]:
if do_local:
    %run {scriptname} 
else:
    !{python_exec} {scriptname}

----------------
Loaded SSD model
----------------
Loading pretrained torchvision's model..
-------
Loaded!
-------
2021-02-11 Running benchmark on host MacBook-Air-de-jeremie.local with cpu
0


## Create a Dataset based on a criterion of the PASCAL VOC project dataset (here animated)

In [14]:
scriptname = 'detect_data_classif.py'

In [15]:
%%writefile {scriptname}

from DCNN_dataset_maker.models import *

i = 0 
suppress=None

for i_image, (data, label) in enumerate(image_dataset_SSD):
    
        # Forward prop.
        predicted_locs, predicted_scores = model(data.unsqueeze(0).to(device)) # Move to default device

        # Detect objects in SSD output
        det_boxes, det_labels, det_scores = model.detect_objects(predicted_locs, predicted_scores, min_score=min_score,
                                                                 max_overlap=max_overlap, top_k=top_k)

        # Decode class integer labels
        det_labels = [rev_label_map[l] for l in det_labels[0].to('cpu').tolist()]

        # Sort by the labels
        is_anime = False
        for lab_ in det_labels :
            if lab_ in animated:
                is_anime = True
                
        if is_anime : 
            i+=1
            print(image_dataset.imgs[i_image][0], is_anime)
            shutil.copy(image_dataset.imgs[i_image][0], path_out_a)
        else:
            print(image_dataset.imgs[i_image][0], is_anime)
            shutil.copy(image_dataset.imgs[i_image][0], path_out_b)
print(i)

Overwriting detect_data_classif.py


In [16]:
if do_local:
    %run {scriptname}
else:
    !{python_exec} {scriptname}

/Users/jjn/Nextcloud/2021_StageM2_Jean-Nicolas/dev/2021-01-19_YOLO_notebook/2021-01-20_a-PyTorch-Tutorial-to-Object-Detection/model.py:496: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  condition = torch.tensor(condition, dtype=torch.uint8).to(device)
/Users/jjn/Nextcloud/2021_StageM2_Jean-Nicolas/dev/2021-01-19_YOLO_notebook/2021-01-20_a-PyTorch-Tutorial-to-Object-Detection/model.py:504: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:25.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/Users/jjn/Nextcloud/2021_StageM2_Jean-Nicolas/dev/2021-01-19_YOLO_notebook/2021-01-20_a-PyTorch-Tutorial-to-Object-Detection/model.py:506: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torc

/Users/jjn/Desktop/test/det/1332.jpg True
/Users/jjn/Desktop/test/det/1357.jpg False
/Users/jjn/Desktop/test/det/1388.jpg True
/Users/jjn/Desktop/test/det/1403.jpg False
/Users/jjn/Desktop/test/det/1412.jpg False
2


## Create a Dataset croped on a class based on the PASCAL VOC project dataset

In [17]:
scriptname = 'detect_crop.py'

In [18]:
%%writefile {scriptname}

from DCNN_dataset_maker.models import *

for i_image, (data, label) in enumerate(image_dataset_SSD):
    # Forward prop.
    predicted_locs, predicted_scores = model(data.unsqueeze(0).to(device)) # Move to default device

    # Detect objects in SSD output
    det_boxes, det_labels, det_scores = model.detect_objects(predicted_locs, predicted_scores, min_score=min_score,
                                                             max_overlap=max_overlap, top_k=top_k)
    # Move detections to the CPU
    det_boxes = det_boxes[0].to('cpu')

    # Transform to original image dimensions
    original_image = Image.open(image_dataset.imgs[i_image][0], mode='r')
    original_dims = torch.FloatTensor(
        [original_image.width, original_image.height, original_image.width, original_image.height]).unsqueeze(0)
    det_boxes = det_boxes * original_dims

    # Decode class integer labels
    det_labels = [rev_label_map[l] for l in det_labels[0].to('cpu').tolist()]

    # If no objects found, the detected labels will be set to ['0.'], i.e. ['background'] in SSD300.detect_objects() in model.py
    annotated_images = []
    i=0
    for x in det_labels:
        if x == class_crop :
            left =  float(det_boxes[i][0])
            upper = float(det_boxes[i][1])
            right = float(det_boxes[i][2])
            lower = float(det_boxes[i][3])
            annotated_image = original_image.crop((left, upper, right, lower))
            i += 1
            print(f'There is {i} {class_crop}(s) in {image_dataset.imgs[i_image][0]}.')
            annotated_image.save(f'{path_out_crop}/{i_image}_{i}.jpg')
        else:
             print (f"No {class_crop} in {image_dataset.imgs[i_image][0]}")        

Overwriting detect_crop.py


In [19]:
if do_local:
    %run {scriptname}
else:
    !{python_exec} {scriptname}

There is 1 person(s) in /Users/jjn/Desktop/test/det/1332.jpg.
No person in /Users/jjn/Desktop/test/det/1332.jpg
No person in /Users/jjn/Desktop/test/det/1357.jpg
There is 1 person(s) in /Users/jjn/Desktop/test/det/1388.jpg.
No person in /Users/jjn/Desktop/test/det/1388.jpg
No person in /Users/jjn/Desktop/test/det/1403.jpg
No person in /Users/jjn/Desktop/test/det/1403.jpg
No person in /Users/jjn/Desktop/test/det/1403.jpg
No person in /Users/jjn/Desktop/test/det/1412.jpg


## Create a dataset centered on a class based on the PASCAL VOC project dataset

In [20]:
scriptname = 'detect_roll.py'

In [21]:
%%writefile {scriptname}

from DCNN_dataset_maker.models import *


for i_image, (data, label) in enumerate(image_dataset_SSD):
    
    # Forward prop.
    predicted_locs, predicted_scores = model(data.unsqueeze(0).to(device)) # Move to default device


    # Detect objects in SSD output
    det_boxes, det_labels, det_scores = model.detect_objects(predicted_locs, predicted_scores, min_score=min_score,
                                                             max_overlap=max_overlap, top_k=top_k)
    
    # Move detections to the CPU
    det_boxes = det_boxes[0].to('cpu')

    # Transform to original image dimensions
    original_image = Image.open(image_dataset.imgs[i_image][0], mode='r')
    N_X = original_image.width # Get the size of the picture
    N_Y = original_image.height
    original_dims = torch.FloatTensor(
        [N_X, N_Y, N_X, N_Y]).unsqueeze(0)
    det_boxes = det_boxes * original_dims

    # Decode class integer labels
    det_labels = [rev_label_map[l] for l in det_labels[0].to('cpu').tolist()]

    # Choose the region of interest as the highest saliency for a given label
    det_score = det_scores[0].tolist()
    ROI = det_score.index(max(det_score)) 
    if class_roll in det_labels:
        if det_labels[ROI] != class_roll:
            det_score[ROI] = 0
            ROI = det_score.index(max(det_score))    

    for x in det_labels:
        if x == class_roll : 
            left =  float(det_boxes[ROI][0]) #Get the position of the bounding box
            upper = float(det_boxes[ROI][1])
            right = float(det_boxes[ROI][2])
            lower = float(det_boxes[ROI][3])
            c1 = int((N_Y//2)-(((lower-upper)//2)+upper)) # "distance" from the center of the picture to the center of the box on the 0 axis
            c2 = int((N_X//2)-(((right-left)//2)+left)) # "distance" from the center of the picture to the center of the box on the 1 axis
            annotated_image = np.roll(original_image, c2, axis=1) # sliding the gaze to the right by moving the picture to the left
            annotated_image = np.roll(annotated_image, c1, axis=0)# sliding the gaze up by moving the picture to the bottom
            print(f'There is a {class_roll}(s) in {image_dataset.imgs[i_image][0]}.')
            annotated_image = Image.fromarray(np.uint8(annotated_image)).save(f'{path_out_roll}/{i_image}.jpg')
        else:
            print (f"No {class_roll} in {image_dataset.imgs[i_image][0]}")        

Overwriting detect_roll.py


In [22]:
if do_local:
    %run {scriptname}
else:
    !{python_exec} {scriptname}

There is a person(s) in /Users/jjn/Desktop/test/det/1332.jpg.
No person in /Users/jjn/Desktop/test/det/1332.jpg
No person in /Users/jjn/Desktop/test/det/1357.jpg
There is a person(s) in /Users/jjn/Desktop/test/det/1388.jpg.
No person in /Users/jjn/Desktop/test/det/1388.jpg
No person in /Users/jjn/Desktop/test/det/1403.jpg
No person in /Users/jjn/Desktop/test/det/1403.jpg
No person in /Users/jjn/Desktop/test/det/1403.jpg
No person in /Users/jjn/Desktop/test/det/1412.jpg
